# Backtest sandbox walkthrough

This notebook accompanies the December 2025 release and demonstrates how to
bootstrap the demo stack, generate a strategy draft with the AI assistant and
run a backtest end-to-end.

## 1. Environment setup

```bash
make demo-up
pip install -r services/algo-engine/requirements.txt
export AI_ASSISTANT_ENABLED=1
export OPENAI_API_KEY="sk-your-key"
```

The optional LangChain/OpenAI dependencies enable the assistant-backed
generation endpoint exposed by the algo engine.

## 2. Bootstrap demo entities

```bash
python scripts/dev/bootstrap_demo.py BTCUSDT 0.25 --order-type market
```

The script provisions an account, entitlements, strategy, order, report and
alert. Copy the JSON identifiers for the next steps.

## 3. Request an AI draft

```python
import httpx

payload = {"prompt": "Mean reversion on BTC 15m", "preferred_format": "yaml"}
resp = httpx.post("http://localhost:8014/strategies/generate", json=payload)
resp.raise_for_status()
draft = resp.json()
draft
```

Review the generated YAML/Python and adjust parameters before importing.

## 4. Import and backtest

```python
strategy_payload = {
    "format": draft["preferred_format"],
    "content": draft["content"],
    "name": draft.get("summary", "AI Draft")
}
created = httpx.post("http://localhost:8014/strategies/import", json=strategy_payload)
created.raise_for_status()
strategy = created.json()
backtest = httpx.post("http://localhost:8014/strategies/{strategy_id}/backtest".format(strategy_id=strategy["id"]),
                        json={"initial_balance": 10000, "market_data": []})
backtest.raise_for_status()
backtest.json()
```

Inspect the files saved under `data/backtests/` for equity curves and logs.

## 5. Tear down

```bash
make demo-down
```

Clean up containers once the walkthrough is complete.